# Model

In [ ]:
!pip install transformers
!pip install bert-tensorflow
#!pip install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tqdm boto3 requests regex sentencepiece sacremoses
#!pip install --upgrade --pre mmf
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install madgrad

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 33.8 MB/s 
     |████████████████████████████████| 895 kB 46.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 64 kB 1.7 MB/s 
     |████████████████████████████████| 132 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 32.1 MB/s 
     |████████████████████████████████| 8.7 MB 33.2 MB/s 
     |████████████████████████████████| 79 kB 6.5 MB/s 
     |████████████████████████████████| 138 kB 36.4 MB/s 
     |████████████████████████████████| 127 kB 39.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERRO

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
import json
import os
from collections import Counter
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from madgrad import MADGRAD

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

import clip
import pickle

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import copy

import transformers

from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    MMBTConfig,
    MMBTModel,
    MMBTForClassification,
    get_linear_schedule_with_warmup,
)

#Create a variable with available device, which will do all needed computations. We will need a GPU, so our device is CUDA.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
#Here is the path for the images
train_img = !unzip gdrive/MyDrive/training.zip

In [ ]:
#upload the dataset with file name, labels and text transcription
data_train = pd.read_excel('memes.xlsx')
data_train

,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription;;;;;;;;;;;;;
0,1.jpg,0,0,0,0,0,Milk Milk.zip;;;;;;;;;;;;;
1,10.jpg,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,0,0,0,0,MAN SEEKING WOMAN Ignad 18 O;;;;;;;;;;;;;
4,10006.jpg,0,0,0,0,0,Me explaining the deep lore of. J.R.R. Tolkein...
...,...,...,...,...,...,...,...
9995,15002.jpg,0,0,0,0,0,WAITING FOR THE END OF THE COVID imgflip.com;...
9996,15003.jpg,0,0,0,0,0,SMART WOMEN ARE AROUND imgflip.com;;;;;;;;;;;;;
9997,15004.jpg,0,0,0,0,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com;...
9998,15005.jpg,0,0,0,0,0,COOKING FOR MY WIFE imgflip.com;;;;;;;;;;;;;


In [ ]:
#let's keep only the 'misogynous' column, as it is the first class that we are going to train
data_train = data_train.drop(['stereotype'], axis=1)
data_train = data_train.drop(['shaming'], axis=1)
data_train = data_train.drop(['objectification'], axis=1)
data_train = data_train.drop(['violence'], axis=1)
data_train = data_train.rename(columns={'Text Transcription;;;;;;;;;;;;;': 'text'})
data_train = data_train.rename(columns={'file_name': 'img'})
data_train = data_train.rename(columns={'misogynous': 'label'})
#let's create the ID column
data_train['id'] = data_train.index

#let's do the same for the validation dataset
#data_val = data_val.drop(['data_type'], axis=1)
#data_val = data_val.rename(columns={'Text Transcription;;;;;;;;;;;;;': 'text'})
#data_val = data_val.rename(columns={'file_name': 'img'})
#data_val = data_val.rename(columns={'misogynous': 'label'})
#id column
#data_val['id'] = data_val.index

In [ ]:
#split train-val
from sklearn.model_selection import train_test_split

# Split dataset in traning and validation(test)
X_train, X_val, Y_train, Y_val = train_test_split(
    data_train.index.values,
    data_train.label.values,
    test_size=0.10,
    random_state=17,
    stratify=data_train.label.values
)


In [ ]:
# Check datasets composition
data_train['data_type'] = ['not_set'] * data_train.shape[0]
data_train.loc[X_train, 'data_type'] = 'train'
data_train.loc[X_val, 'data_type'] = 'val'
data_train.groupby(['label', 'data_type']).count()

img  text    id
label data_type                  
0     train      4500  4500  4500
      val         500   500   500
1     train      4500  4500  4500
      val         500   500   500

In [ ]:
data_train.to_excel('split_train_val.xlsx')

In [ ]:
data = pd.read_excel('split_train_val.xlsx')
data

,Unnamed: 0,img,label,text,id,data_type
0,0,1.jpg,0,Milk Milk.zip;;;;;;;;;;;;;,0,train
1,1,10.jpg,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S...",1,train
2,2,1000.jpg,0,BREAKING NEWS: Russia releases photo of DONALD...,2,val
3,3,10000.jpg,0,MAN SEEKING WOMAN Ignad 18 O;;;;;;;;;;;;;,3,train
4,4,10006.jpg,0,Me explaining the deep lore of. J.R.R. Tolkein...,4,train
...,...,...,...,...,...,...
9995,9995,15002.jpg,0,WAITING FOR THE END OF THE COVID imgflip.com;...,9995,train
9996,9996,15003.jpg,0,SMART WOMEN ARE AROUND imgflip.com;;;;;;;;;;;;;,9996,train
9997,9997,15004.jpg,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com;...,9997,train
9998,9998,15005.jpg,0,COOKING FOR MY WIFE imgflip.com;;;;;;;;;;;;;,9998,train


In [ ]:
data = data.dropna() #2 rows dropped
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
train = data.loc[data['data_type'] == 'train']
val = data.loc[data['data_type'] == 'val']

In [ ]:
# save json file for train
train.to_json('train_mis.json', orient='records', lines=True)
# same for validation
val.to_json('val_mis.json', orient='records', lines=True)

In [ ]:
#now we drag 'train.json' and 'val.json' into the folder 'dataset'. Let's have a look.
data_train = pd.read_json('/content/dataset/train_mis.json', lines=True)
data_train = data_train.drop('data_type', axis=1)
data_train

,img,label,text,id
0,1.jpg,0,Milk Milk.zip;;;;;;;;;;;;;,0
1,10.jpg,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S...",1
2,10000.jpg,0,MAN SEEKING WOMAN Ignad 18 O;;;;;;;;;;;;;,3
3,10006.jpg,0,Me explaining the deep lore of. J.R.R. Tolkein...,4
4,10007.jpg,0,PICTOPHLE APP *Straight white malle starts tal...,5
...,...,...,...,...
8994,15002.jpg,0,WAITING FOR THE END OF THE COVID imgflip.com;...,9995
8995,15003.jpg,0,SMART WOMEN ARE AROUND imgflip.com;;;;;;;;;;;;;,9996
8996,15004.jpg,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com;...,9997
8997,15005.jpg,0,COOKING FOR MY WIFE imgflip.com;;;;;;;;;;;;;,9998


In [ ]:
#let's see the val dataset as well
data_val = pd.read_json('/content/dataset/val_mis.json', lines=True)
data_val = data_val.drop('data_type', axis=1)
data_val

,img,label,text,id
0,1000.jpg,0,BREAKING NEWS: Russia releases photo of DONALD...,2
1,10016.jpg,1,IATE A WHOLE PIZZA HUT Posted By: Jorge Kerby ...,14
2,10022.jpg,0,HM.. NEXT GAME! ARE YOU WINNING LOVE? 6000 ......,19
3,10043.jpg,1,"When Feminists realize that most of them ""MENs...",37
4,10048.jpg,0,Women: I hate my hair. No products work for me...,41
...,...,...,...,...
994,994.jpg,1,Is Maths related to Science?;;;;;;;;;;;;;,9938
995,9949.jpg,0,SFR 11:00 When I watch anime with my french we...,9947
996,9961.jpg,0,SHE SAID HI TO ME serenayaraf PRO 2h @der_urs ...,9959
997,9962.jpg,1,When the pussy so good you lose all track of t...,9960


In [ ]:
#let's move also the folder TRAINING into dataset and then execute this
!mv dataset/TRAINING/*jpg dataset/

In [ ]:
a#Load CLIP model and needed preprocessing.

clip_model, preprocess = clip.load("RN50x4", device=device, jit=False)

#Freeze weights of CLIP feature encoder, as we will not finetune it.

for p in clip_model.parameters():
    p.requires_grad = False

#Initialize needed variables. 

num_image_embeds = 4
num_labels = 1
gradient_accumulation_steps = 20
data_dir = '/content/dataset'
max_seq_length = 80 
max_grad_norm = 0.5
train_batch_size = 16
eval_batch_size = 16
image_encoder_size = 288
image_features_size = 640
num_train_epochs = 5


#Create a function that will prepare an image for CLIP encoder in a special manner. 
#This function will split image into three tiles (by height or width, depending on the aspect ratio of the image). 
#Finally we will get four vectors after encoding (one vector for each tile and one vector for whole image that was padded to square).

def slice_image(im, desired_size):
    '''
    Resize and slice image
    '''
    old_size = im.size  

    ratio = float(desired_size)/min(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = im.resize(new_size, Image.ANTIALIAS)
    
    ar = np.array(im)
    images = []
    if ar.shape[0] < ar.shape[1]:
        middle = ar.shape[1] // 2
        half = desired_size // 2
        
        images.append(Image.fromarray(ar[:, :desired_size]))
        images.append(Image.fromarray(ar[:, middle-half:middle+half]))
        images.append(Image.fromarray(ar[:, ar.shape[1]-desired_size:ar.shape[1]]))
    else:
        middle = ar.shape[0] // 2
        half = desired_size // 2
        
        images.append(Image.fromarray(ar[:desired_size, :]))
        images.append(Image.fromarray(ar[middle-half:middle+half, :]))
        images.append(Image.fromarray(ar[ar.shape[0]-desired_size:ar.shape[0], :]))

    return images
def resize_pad_image(im, desired_size):
    '''
    Resize and pad image to a desired size
    '''
    old_size = im.size  

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = im.resize(new_size, Image.ANTIALIAS)

    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))

    return new_im

#Define a function, that will get image features from CLIP.

class ClipEncoderMulti(nn.Module):
    def __init__(self, num_embeds, num_features=image_features_size):
        super().__init__()        
        self.model = clip_model
        self.num_embeds = num_embeds
        self.num_features = num_features

    def forward(self, x):
        # 4x3x288x288 -> 1x4x640
        out = self.model.encode_image(x.view(-1,3,288,288))
        out = out.view(-1, self.num_embeds, self.num_features).float()
        return out  # Bx4x640

100%|███████████████████████████████████████| 402M/402M [00:06<00:00, 62.8MiB/s]


In [ ]:
class JsonlDataset(Dataset):
    def __init__(self, data_path, tokenizer, transforms, max_seq_length):
        self.data = [json.loads(l) for l in open(data_path)]
        self.data_dir = os.path.dirname(data_path)
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = torch.LongTensor(self.tokenizer.encode(self.data[index]["text"], add_special_tokens=True))
        start_token, sentence, end_token = sentence[0], sentence[1:-1], sentence[-1]
        sentence = sentence[:self.max_seq_length]

        label = torch.FloatTensor([self.data[index]["label"]])

        image = Image.open(os.path.join(self.data_dir, self.data[index]["img"])).convert("RGB")
        sliced_images = slice_image(image, 288)
        sliced_images = [np.array(self.transforms(im)) for im in sliced_images]
        image = resize_pad_image(image, image_encoder_size)
        image = np.array(self.transforms(image))
        
        sliced_images = [image] + sliced_images         
        sliced_images = torch.from_numpy(np.array(sliced_images)).to(device)

        return {
            "image_start_token": start_token,            
            "image_end_token": end_token,
            "sentence": sentence,
            "image": sliced_images,
            "label": label            
        }

    def get_label_frequencies(self):
        label_freqs = Counter()
        for row in self.data:
            label_freqs.update([row["label"]])
        return label_freqs
    
    def get_labels(self):
        labels = []
        for row in self.data:
            labels.append(row["label"])
        return labels

In [ ]:
def collate_fn(batch):
    lens = [len(row["sentence"]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
    text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        text_tensor[i_batch, :length] = input_row["sentence"]
        mask_tensor[i_batch, :length] = 1
    
    img_tensor = torch.stack([row["image"] for row in batch])
    tgt_tensor = torch.stack([row["label"] for row in batch])
    img_start_token = torch.stack([row["image_start_token"] for row in batch])
    img_end_token = torch.stack([row["image_end_token"] for row in batch])

    return text_tensor, mask_tensor, img_tensor, img_start_token, img_end_token, tgt_tensor

In [ ]:
def load_examples(tokenizer, evaluate=False):
    path = os.path.join(data_dir, "val_mis.json" if evaluate else f"train_mis.json")
    transforms = preprocess
    dataset = JsonlDataset(path, tokenizer, transforms, max_seq_length - num_image_embeds - 2)
    return dataset

In [ ]:
def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    
def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [ ]:
model_name = 'bert-base-uncased'
transformer_config = AutoConfig.from_pretrained(model_name) 
transformer = AutoModel.from_pretrained(model_name, config=transformer_config)
img_encoder = ClipEncoderMulti(num_image_embeds)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
config = MMBTConfig(transformer_config, num_labels=num_labels, modal_hidden_size=image_features_size)
model = MMBTForClassification(config, transformer, img_encoder)

In [ ]:
model.to(device);

In [ ]:
train_dataset = load_examples(tokenizer, evaluate=False)
eval_dataset = load_examples(tokenizer, evaluate=True)   

train_sampler = RandomSampler(train_dataset)
eval_sampler = SequentialSampler(eval_dataset)

train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=train_batch_size,
        collate_fn=collate_fn
    )


eval_dataloader = DataLoader(
        eval_dataset, 
        sampler=eval_sampler, 
        batch_size=eval_batch_size, 
        collate_fn=collate_fn
    )

In [ ]:
from tensorflow import keras
from keras import losses

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ["bias", 
            "LayerNorm.weight"
           ]
weight_decay = 0.0005

optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

t_total = (len(train_dataloader) // gradient_accumulation_steps) * num_train_epochs
warmup_steps = t_total // 10

optimizer = MADGRAD(optimizer_grouped_parameters, lr=2e-4)

scheduler = get_linear_schedule_with_warmup(
        optimizer, warmup_steps, t_total
    )

criterion = nn.BCEWithLogitsLoss()

In [ ]:
def evaluate(model, tokenizer, criterion, dataloader, tres = 0.5): 
    
    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    proba = None
    out_label_ids = None
    for batch in dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            labels = batch[5]
            inputs = {
                "input_ids": batch[0],
                "input_modal": batch[2],
                "attention_mask": batch[1],
                "modal_start_tokens": batch[3],
                "modal_end_tokens": batch[4],
                "return_dict": False
            }
            outputs = model(**inputs)
            logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
            tmp_eval_loss = criterion(logits, labels)
            
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = torch.sigmoid(logits).detach().cpu().numpy() > tres
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:            
            preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > tres, axis=0)
            proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
    
    eval_loss = eval_loss / nb_eval_steps

    result = {
        "loss": eval_loss,
        "accuracy": accuracy_score(out_label_ids, preds),
        "AUC": roc_auc_score(out_label_ids, proba),
        "micro_f1": f1_score(out_label_ids, preds, average="micro"),
        "prediction": preds,
        "labels": out_label_ids,
        "proba": proba
    }
    
    return result

In [ ]:
optimizer_step = 0
global_step = 0
train_step = 0
tr_loss, logging_loss = 0.0, 0.0
best_valid_auc = 0.75
best_valid_f1 = 0.70
global_steps_list = []
train_loss_list = []
val_loss_list = []
val_acc_list = []
val_auc_list = []
val_f1_list = []
eval_every = len(train_dataloader) // 7
running_loss = 0
file_path="models/"

model.zero_grad()

for i in range(num_train_epochs):
    print("Epoch", i+1, f"from {num_train_epochs}")
    whole_y_pred=np.array([])
    whole_y_t=np.array([])
    for step, batch in enumerate(tqdm(train_dataloader)):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        labels = batch[5]
        inputs = {
            "input_ids": batch[0],
            "input_modal": batch[2],
            "attention_mask": batch[1],
            "modal_start_tokens": batch[3],
            "modal_end_tokens": batch[4],
            "return_dict": False
        }
        outputs = model(**inputs)
        logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
        loss = criterion(logits, labels)        
        
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
            
        loss.backward()
        
        tr_loss += loss.item()
        running_loss += loss.item()
        global_step += 1
        
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule         
            
            optimizer_step += 1
            optimizer.zero_grad()   
                        
        if (step + 1) % eval_every == 0:
            
            average_train_loss = running_loss / eval_every
            train_loss_list.append(average_train_loss)
            global_steps_list.append(global_step)
            running_loss = 0.0  
            
            val_result = evaluate(model, tokenizer, criterion, eval_dataloader)
            
            val_loss_list.append(val_result['loss'])
            val_acc_list.append(val_result['accuracy'])
            val_auc_list.append(val_result['AUC'])
            val_f1_list.append(val_result['micro_f1'])
            
            # checkpoint
            if val_result['micro_f1'] > best_valid_f1:
                best_valid_f1 = val_result['micro_f1']
                val_loss = val_result['loss']
                val_acc = val_result['accuracy']
                val_f1 = val_result['micro_f1']
                model_path = f'{file_path}/model-embs{num_image_embeds}-seq{max_seq_length}-F1{best_valid_f1:.3f}-loss{val_loss:.3f}-acc{val_acc:.3f}.pt'
                print(f"F1 improved, so saving this model")  
                save_checkpoint(model_path, model, val_result['loss'])             
            
            print("Train loss:", f"{average_train_loss:.4f}", 
                  "Val loss:", f"{val_result['loss']:.4f}",
                  "Val acc:", f"{val_result['accuracy']:.4f}",
                  "F1 score:", f"{val_result['micro_f1']:.4f}")   
    print('\n')

In [ ]:
# path for test files
!unzip gdrive/MyDrive/test.zip -d "/content/testing"

In [ ]:
import pandas as pd
num_labels = 1
data_dir = 'testing/test'
test_batch_size = 16

In [ ]:
class TestJsonlDataset(Dataset):
        def __init__(self, data_path, tokenizer, transforms, max_seq_length):
            self.data = [json.loads(l) for l in open(data_path)]
            self.data_dir = os.path.dirname(data_path)
            self.tokenizer = tokenizer
            self.max_seq_length = max_seq_length
            self.transforms = transforms

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
            sentence = torch.LongTensor(self.tokenizer.encode(self.data[index]["text"], add_special_tokens=True))
            start_token, sentence, end_token = sentence[0], sentence[1:-1], sentence[-1]
            sentence = sentence[:self.max_seq_length]

            id = torch.LongTensor([self.data[index]["id"]])        
            image = Image.open(os.path.join(self.data_dir, self.data[index]["img"])).convert("RGB")
            sliced_images = slice_image(image, 288)
            sliced_images = [np.array(self.transforms(im)) for im in sliced_images]
            image = resize_pad_image(image, image_encoder_size)
            image = np.array(self.transforms(image))        
            sliced_images = [image] + sliced_images        
            sliced_images = torch.from_numpy(np.array(sliced_images)).to(device)

            return {
                "image_start_token": start_token,            
                "image_end_token": end_token,
                "sentence": sentence,
                "image": sliced_images,
                "id": id,
            }

In [ ]:
def final_collate_fn(batch):
        lens = [len(row["sentence"]) for row in batch]
        bsz, max_seq_len = len(batch), max(lens)

        mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
        text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

        for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
            text_tensor[i_batch, :length] = input_row["sentence"]
            mask_tensor[i_batch, :length] = 1

        img_tensor = torch.stack([row["image"] for row in batch])
        id_tensor = torch.stack([row["id"] for row in batch])
        img_start_token = torch.stack([row["image_start_token"] for row in batch])
        img_end_token = torch.stack([row["image_end_token"] for row in batch])

        return text_tensor, mask_tensor, img_tensor, img_start_token, img_end_token, id_tensor

In [ ]:
def load_test_examples(test_file="test_seen (1).jsonl"):
        path = os.path.join(data_dir, test_file)
        dataset = TestJsonlDataset(path, tokenizer, preprocess, max_seq_length - num_image_embeds - 2)
        return dataset

In [ ]:
def final_prediction(model, dataloader): 
        preds = None
        proba = None
        all_ids = None
        for batch in tqdm(dataloader):
            model.eval()
            batch = tuple(t.to(device) for t in batch)
            with torch.no_grad():
                ids = batch[5]
                inputs = {
                    "input_ids": batch[0],
                    "input_modal": batch[2],
                    "attention_mask": batch[1],
                    "modal_start_tokens": batch[3],
                    "modal_end_tokens": batch[4],
                    "return_dict": False
                }
                outputs = model(**inputs)
                logits = outputs[0]
            if preds is None:
                all_ids = ids.detach().cpu().numpy()
                preds = torch.sigmoid(logits).detach().cpu().numpy() > 0.5
                proba = torch.sigmoid(logits).detach().cpu().numpy()            
            else:  
                all_ids = np.append(all_ids, ids.detach().cpu().numpy(), axis=0)
                preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > 0.5, axis=0)
                proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)

        result = {
            "ids": all_ids,
            "preds": preds,
            "probs": proba,
        }

        return result

In [ ]:
!mv "/content/test_seen (2).jsonl" "/content/testing/test"

In [ ]:
final_test = load_test_examples()
final_test_sampler = SequentialSampler(final_test)
final_test_dataloader = DataLoader(
            final_test, 
            sampler=final_test_sampler, 
            batch_size=test_batch_size, 
            collate_fn=final_collate_fn
        )

In [ ]:
results = final_prediction(model, final_test_dataloader)

results['ids'] = results['ids'].reshape(-1)
results['preds'] = results['preds'].reshape(-1)
results['probs'] = results['probs'].reshape(-1)

df = pd.DataFrame(results)
df = df[['ids', 'probs', 'preds']]
df.columns = ['id', 'proba', 'label']
df.label = df.label.astype(int)

df.to_excel('clip_stereo_pred.xlsx', index=False, float_format='%.3f')


  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
df.to_excel('clip_stereo_pred.xlsx', index=False, float_format='%.3f')